In [1]:
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import warnings
import glob 
import os 
from eli5.sklearn import PermutationImportance
import eli5
from sklearn.metrics import make_scorer
import shap

# Make NumPy printouts easier to read.
np.set_printoptions(precision=5, suppress=True)
warnings.filterwarnings("ignore")

print(tf.__version__)

2.10.0


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
DATASET_CALOSC=pd.read_csv('../Dane/pogoda/Obrobione.csv')
DATASET_CALOSC=DATASET_CALOSC.drop(['przekaźnik pokojowy','praca_pieca','temperatura wewnętrzna na kominku','temperatura wewnętrzna nad kuchenką gazową'],axis=1)
DATASET_CALOSC

,temperatura zewnętrzna,wilgotność zewnętrzna,ciśnienie atmosferyczne,prędkość wiatru,kierunek wiatru,wilgotność wewnętrzna,temperatura wewnętrzna
0,0.000000,92.200000,972.80000,2.200000,80.00000,37.250000,22.1875
1,0.000000,92.000000,972.80000,3.200000,92.00000,37.250000,22.1875
2,0.000000,92.000000,972.80000,3.300000,92.00000,37.000000,22.1875
3,0.000000,91.900000,972.80000,3.300000,84.00000,37.000000,22.1875
4,0.000000,92.100000,972.80000,2.900000,98.00000,37.000000,22.2500
...,...,...,...,...,...,...,...
78093,14.674074,40.170370,999.50000,1.423077,214.00000,33.560127,21.8750
78094,14.774074,39.485185,999.50000,1.553846,303.07692,33.689873,21.8750
78095,14.725926,38.533333,999.42593,1.600000,185.69231,33.750000,21.8750
78096,14.625926,38.596296,999.40000,1.215385,163.76923,33.560127,21.8750


In [4]:
x = DATASET_CALOSC.iloc[:, :6]  
y = DATASET_CALOSC.iloc[:, 6]   

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(x)

In [5]:
def custom_scheduler(epoch, lr):
    if epoch < 4:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


model = tf.keras.models.Sequential([
    normalizer,
          tf.keras.layers.Dense(512, activation='relu'),
          tf.keras.layers.Dense(256, activation='relu'),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(32, activation='relu'),
          tf.keras.layers.Dense(1,activation='linear') 
])

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss="mse",
    metrics=['mae'])

scheduler = LearningRateScheduler(custom_scheduler)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1)
callbacks = [scheduler, tensorboard]

EPOCHS = 50
BATCH_SIZE=32

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks)

Epoch 1/50
1953/1953 [==============================] - 6s 2ms/step - loss: 5.0064 - mae: 0.7812 - lr: 0.0010
Epoch 2/50
1953/1953 [==============================] - 5s 2ms/step - loss: 0.1622 - mae: 0.3146 - lr: 0.0010
Epoch 3/50
1953/1953 [==============================] - 5s 2ms/step - loss: 0.1557 - mae: 0.3080 - lr: 0.0010
Epoch 4/50
1953/1953 [==============================] - 5s 2ms/step - loss: 0.1415 - mae: 0.2932 - lr: 0.0010
Epoch 5/50
1953/1953 [==============================] - 5s 2ms/step - loss: 0.1226 - mae: 0.2719 - lr: 9.0484e-04
Epoch 6/50
1953/1953 [==============================] - 5s 3ms/step - loss: 0.1074 - mae: 0.2543 - lr: 8.1873e-04
Epoch 7/50
1953/1953 [==============================] - 5s 2ms/step - loss: 0.0993 - mae: 0.2444 - lr: 7.4082e-04
Epoch 8/50
1953/1953 [==============================] - 5s 2ms/step - loss: 0.0868 - mae: 0.2271 - lr: 6.7032e-04
Epoch 9/50
1953/1953 [==============================] - 5s 2ms/step - loss: 0.0808 - mae: 0.2188 - lr: 6

In [7]:
model.evaluate(x_test, y_test)

489/489 [==============================] - 1s 2ms/step - loss: 0.0256 - mae: 0.1136


[0.025624247267842293, 0.11359873414039612]

# Permutation Feature Importance

In [8]:
def model_score(x, y):
    return mean_squared_error(y, model.predict(x))

In [9]:
scorer = make_scorer(model_score, greater_is_better=True)

perm = PermutationImportance(model, scoring=scorer, random_state=1).fit(x_test, y_test)


489/489 [==============================] - 0s 909us/step


In [10]:
eli5.show_weights(perm, feature_names = x_test.columns.tolist())

Weight,Feature
65.6234 ± 1.8697,wilgotność zewnętrzna
28.8867 ± 1.2035,wilgotność wewnętrzna
28.4172 ± 1.5971,temperatura zewnętrzna
17.3586 ± 1.7198,ciśnienie atmosferyczne
4.7781 ± 0.7175,prędkość wiatru
-4.1836 ± 0.4537,kierunek wiatru


# SHapley Additive exPlanations

In [11]:
# Przekształć dane do formatu NumPy, jeśli są w Pandas DataFrame
x_train_array = np.array(x_train)
x_test_array = np.array(x_test)

# Utwórz obiekt explainera SHAP
explainer = shap.DeepExplainer(model, x_train_array)

# Oblicz wartości SHAP dla testowego zestawu danych
shap_values = explainer.shap_values(x_test_array)

# # Wizualizacja
shap.summary_plot(shap_values, x_test_array)

TypeError: 'NoneType' object cannot be interpreted as an integer